In [1]:
import os
from typing import Final

from bayes_opt import BayesianOptimization
from bayes_opt.event import Events
from bayes_opt.logger import JSONLogger
from bayes_opt.util import load_logs
from numpy import where
from pandas import DataFrame, Series, concat, read_csv, set_option
from sklearn.metrics import f1_score
from sklearn.svm import OneClassSVM

set_option("display.max_columns", None)
NUM_TRIALS: Final[int] = 5
LOGS_PATH: Final[str] = "../reports/logs_bayesian.log"
logger = JSONLogger(path=LOGS_PATH, reset=False)

In [2]:
X_train = read_csv("../data/PAMAP2/x_train_data.csv")
X_test = read_csv("../data/PAMAP2/x_test_data.csv")
y_train = read_csv("../data/PAMAP2/y_train_data.csv")
y_test = read_csv("../data/PAMAP2/y_test_data.csv")

X_train["activity"] = y_train  # First 80% of the data
X_test["activity"] = y_test  # Last 20% of the data

# MIN_SAMPLES = X_train["activity"].value_counts().min()
MIN_SAMPLES = X_train["activity"].value_counts().sort_values().iloc[0]
MAXIMAZED = False

models: dict[int, dict] = {}
training_data: DataFrame
testing_data: DataFrame
train_targets: Series
test_targets: Series

In [3]:
def objective_function(nu: float, gamma: float) -> float:
    """
    Objective function to optimize F1-Score on the train and test set.
    Evaluates the OneClassSVM model using stratified k-fold cross-validation.

    Args:
        nu (float): nu param to evaluate.
        gamma (float): gamma param to evaluate.

    Returns:
        float: F1-Score on the test set of this iteration.
    """
    oc_svm = OneClassSVM(kernel="rbf", nu=nu, gamma=gamma).fit(training_data)
    return f1_score(
        test_targets, where(oc_svm.predict(testing_data) == -1, True, False)
    )


optimizer = BayesianOptimization(
    objective_function, {"nu": (0.01, 0.5), "gamma": (1e-4, 1e-1)}, random_state=42
)
if not os.path.exists(LOGS_PATH):
    with open(LOGS_PATH, "w") as fp:
        pass
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [4]:
for i in range(1, len((activities := X_train["activity"].unique()))):
    training = (  # picks the first n samples of each class
        X_train[X_train["activity"].isin(activities[:i])]
        .groupby("activity")
        .head(MIN_SAMPLES)
        .copy()
    )
    testing = X_test[X_test["activity"] == activities[i]].head(MIN_SAMPLES)
    training.loc[:, "isNovelty"], testing.loc[:, "isNovelty"] = False, True
    novelty = concat(
        [testing, training.sample(n=int(0.2 * len(training)), random_state=42)]
    )
    training_data = training.drop(columns=["isNovelty"])
    train_targets = training["isNovelty"]
    testing_data = novelty.drop(columns=["isNovelty"])
    test_targets = novelty["isNovelty"]  # only current activity (as novelty)

    load_logs(optimizer, logs=[LOGS_PATH])
    print("New optimizer is now aware of {} points.".format(len(optimizer.space)))

    if not MAXIMAZED:
        optimizer.maximize(init_points=5, n_iter=25)
        MAXIMAZED = True

    else:
        load_logs(optimizer, logs=[LOGS_PATH])
        print(f"Already maximized, sugesting new {NUM_TRIALS} points")
        for i in range(NUM_TRIALS):
            MAX = optimizer.max["target"]
            print(
                "Next point:",
                next_point_to_probe := optimizer.suggest(),
                "Found the target:",
                target := objective_function(**next_point_to_probe),
            )
            optimizer.register(params=next_point_to_probe, target=target)
            if target > MAX:
                print("New best points found, continuing optimization")
                i = 0

    models[i] = optimizer.max

New optimizer is now aware of 0 points.
New optimizer is now aware of 30 points.
Already maximized, sugesting new 5 points
Next point: {'gamma': np.float64(0.0797722007210874), 'nu': np.float64(0.1108306138605744)} Found the target: 0.9661633109619687
Next point: {'gamma': np.float64(0.05160167921942936), 'nu': np.float64(0.1875677929648555)} Found the target: 0.9582582166135072
Next point: {'gamma': np.float64(0.0002955484147134074), 'nu': np.float64(0.34431722734339826)} Found the target: 0.933909987836194
Next point: {'gamma': np.float64(0.046088902356152865), 'nu': np.float64(0.0824802216728603)} Found the target: 0.9702330805953384
Next point: {'gamma': np.float64(0.07901767965803573), 'nu': np.float64(0.04652082563466857)} Found the target: 0.9665687508742481
New optimizer is now aware of 35 points.
Already maximized, sugesting new 5 points
Next point: {'gamma': np.float64(0.02160347581782693), 'nu': np.float64(0.4223263983407976)} Found the target: 0.8903374233128835
Next point:

In [5]:
models

{1: {'target': np.float64(0.9985224586288416),
  'params': {'gamma': np.float64(0.00010145920160011348),
   'nu': np.float64(0.016972678234102926)}},
 4: {'target': np.float64(0.9985224586288416),
  'params': {'gamma': np.float64(0.00010145920160011348),
   'nu': np.float64(0.016972678234102926)}}}